In [1]:
import copy
import os
import pickle
import shlex
import time
from datetime import datetime
from subprocess import PIPE, Popen

import BeamWithSprings as bm
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ajf_bridge_fem import *
from pandarallel import pandarallel
from scipy import stats

import ajf_plts

SEED = ajf_plts.get_seed("6_process_51_years.ipynb")

plt.style.use(["./ajf_plts/base.mplstyle", "./ajf_plts/legend_frame.mplstyle"])

# Load input vehicle and temperature data

In [2]:
in_df = pd.read_parquet("./vehicle_loads.parquet")
in_df

,seq_month,year,month,day,hour,minute,dayofweek,air_temperature,freq,E_val,...,W4,W5,W6,S0,S1,S2,S3,S4,S5,speed
0,0,0,0,1,1,36,6,4.880000,5.786878,2.883737e+10,...,41187.930,0.000,0.00,0.0,3.75,9.60,10.85,NaN,NaN,23.0
1,0,0,0,1,4,43,6,3.555000,5.795990,2.892656e+10,...,0.000,0.000,0.00,0.0,4.50,5.85,NaN,NaN,NaN,24.0
2,0,0,0,1,5,10,6,3.216667,5.799653,2.896242e+10,...,0.000,0.000,0.00,0.0,5.20,6.55,NaN,NaN,NaN,28.0
3,0,0,0,1,5,12,6,3.200000,5.799898,2.896483e+10,...,0.000,0.000,0.00,0.0,5.95,7.45,NaN,NaN,NaN,27.5
4,0,0,0,1,6,57,6,3.655000,5.801216,2.897772e+10,...,0.000,0.000,0.00,0.0,6.05,NaN,NaN,NaN,NaN,25.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6116953,731,60,11,31,22,16,4,12.200000,5.598480,2.699307e+10,...,32361.945,33342.610,0.00,0.0,3.80,10.25,11.50,12.8,NaN,24.5
6116954,731,60,11,31,22,37,4,12.200000,5.599069,2.699884e+10,...,75511.205,0.000,0.00,0.0,5.45,13.75,15.00,NaN,NaN,23.5
6116955,731,60,11,31,22,58,4,12.200000,5.599659,2.700461e+10,...,0.000,0.000,0.00,0.0,6.40,NaN,NaN,NaN,NaN,31.5
6116956,731,60,11,31,23,7,4,12.200000,5.599715,2.700516e+10,...,45110.590,44129.925,45110.59,0.0,2.60,3.90,9.70,11.0,12.3,25.5


# Config

In [6]:
N_procs = 8
N_splits = N_procs * 10


# MODEL = "LB"
# DELTA_MIN = 0.0
# DELTA_MAX = 0.2
# DELTA_STEPS = 0.02
# X_DAM = LB_span_length / 3.0

MODEL = "GH"
DELTA_MIN = 0.0
DELTA_MAX = 0.25
DELTA_STEPS = 0.05
X_DAM = GH_side_span_length + GH_center_span_length / 3.0

run_id = f"{datetime.now().isoformat()}_{MODEL}_{DELTA_STEPS}-step-delta"

In [7]:
if DELTA_STEPS == "coarse":
    delta = np.array([0.0, 0.01, 0.025, 0.05, 0.1, 0.2])
elif type(DELTA_STEPS) == float:
    delta = np.arange(DELTA_MIN, DELTA_MAX + 0.5 * DELTA_STEPS, DELTA_STEPS)
else:
    raise Exception("DELTA_STEPS is incorrect type")

in_df["x_dam"] = X_DAM

dfs = []
for d in delta:
    in_df["delta"] = d
    dfs.append(in_df.copy())

df = pd.concat(dfs).reset_index(drop=True)

del dfs

df.head(), df.tail()

(   seq_month  year  month  day  hour  minute  dayofweek  air_temperature  \
 0          0     0      0    1     1      36          6         4.880000   
 1          0     0      0    1     4      43          6         3.555000   
 2          0     0      0    1     5      10          6         3.216667   
 3          0     0      0    1     5      12          6         3.200000   
 4          0     0      0    1     6      57          6         3.655000   
 
        freq         E_val  ...   W6   S0    S1    S2     S3  S4  S5  speed  \
 0  5.786878  2.883737e+10  ...  0.0  0.0  3.75  9.60  10.85 NaN NaN   23.0   
 1  5.795990  2.892656e+10  ...  0.0  0.0  4.50  5.85    NaN NaN NaN   24.0   
 2  5.799653  2.896242e+10  ...  0.0  0.0  5.20  6.55    NaN NaN NaN   28.0   
 3  5.799898  2.896483e+10  ...  0.0  0.0  5.95  7.45    NaN NaN NaN   27.5   
 4  5.801216  2.897772e+10  ...  0.0  0.0  6.05   NaN    NaN NaN NaN   25.5   
 
        x_dam  delta  
 0  17.733333    0.0  
 1  17.733333 

In [9]:
splits = np.array_split(df, N_splits)

In [10]:
if MODEL == "LB":
    beam = construct_model_LB()
    dofs = [beam.x_pos_to_rot_dof(x) for x in [0, LB_span_length]]
    dof_names = ["L", "R"]
elif MODEL == "GH":
    beam = construct_model_GH()
    dofs = [
        beam.x_pos_to_rot_dof(x)
        for x in [
            0,
            GH_side_span_length,
            GH_side_span_length + GH_center_span_length,
            GH_bridge_length,
        ]
    ]
    dof_names = ["L_abut", "L_pier", "R_pier", "R_abut"]

temp_data_path = f"./temp_data/{run_id}"
args_file = f"{temp_data_path}/temp_parallel_arguments.txt"

In [11]:
# check directory exists
if not os.path.exists(temp_data_path):
    os.makedirs(temp_data_path)
else:
    error("already exists")

with open(args_file, "w") as f:
    for i, sp in enumerate(splits):
        sp_file = f"{temp_data_path}/temp_{i}.feather"
        f.write(
            f"{run_id}\t{i}\t{sp_file}\t{dofs}\t{dof_names}\t{MODEL}\t{temp_data_path}\n"
        )
        sp.reset_index(drop=True).to_feather(sp_file)

In [12]:
del in_df
del splits

In [13]:
start = time.time()
with Popen(
    shlex.split(
        f"parallel -j {N_procs} -a {args_file} --colsep '\t' export OMP_NUM_THREADS=1 && python 6a_process_block.py"
    )
) as p:
    p.wait()
    end = time.time()

print(f"Finished in {end - start}s")

3: finished processing in 1473.7105782032013s
3: saved to ./temp_data/2023-02-05T19:45:04.032244_GH_0.05-step-delta/2023-02-05T19:45:04.032244_GH_0.05-step-delta_3_results.feather, shutting down
2: finished processing in 1481.5665669441223s
2: saved to ./temp_data/2023-02-05T19:45:04.032244_GH_0.05-step-delta/2023-02-05T19:45:04.032244_GH_0.05-step-delta_2_results.feather, shutting down
5: finished processing in 1483.4402492046356s
5: saved to ./temp_data/2023-02-05T19:45:04.032244_GH_0.05-step-delta/2023-02-05T19:45:04.032244_GH_0.05-step-delta_5_results.feather, shutting down
1: finished processing in 1483.6294014453888s
1: saved to ./temp_data/2023-02-05T19:45:04.032244_GH_0.05-step-delta/2023-02-05T19:45:04.032244_GH_0.05-step-delta_1_results.feather, shutting down
4: finished processing in 1488.6434864997864s
4: saved to ./temp_data/2023-02-05T19:45:04.032244_GH_0.05-step-delta/2023-02-05T19:45:04.032244_GH_0.05-step-delta_4_results.feather, shutting down
6: finished processing in

# Join results 

In [18]:
results_df = pd.concat(
    [
        pd.read_feather(f"{temp_data_path}/{run_id}_{i}_results.feather")
        for i in np.arange(N_splits)
    ]
).reset_index(drop=True)
results_df.to_parquet(f"./{run_id}", partition_cols=["delta", "year", "month"])

In [19]:
results_df

,seq_month,year,month,day,hour,minute,dayofweek,air_temperature,freq,E_val,...,S3,S4,S5,speed,x_dam,delta,L_abut,L_pier,R_pier,R_abut
0,0,0,0,1,1,36,6,4.880000,5.786878,2.883737e+10,...,10.85,NaN,NaN,23.0,17.733333,0.0,0.000138,0.000065,0.000062,0.000135
1,0,0,0,1,4,43,6,3.555000,5.795990,2.892656e+10,...,NaN,NaN,NaN,24.0,17.733333,0.0,0.000257,0.000116,0.000116,0.000246
2,0,0,0,1,5,10,6,3.216667,5.799653,2.896242e+10,...,NaN,NaN,NaN,28.0,17.733333,0.0,0.000117,0.000055,0.000055,0.000113
3,0,0,0,1,5,12,6,3.200000,5.799898,2.896483e+10,...,NaN,NaN,NaN,27.5,17.733333,0.0,0.000230,0.000097,0.000099,0.000221
4,0,0,0,1,6,57,6,3.655000,5.801216,2.897772e+10,...,NaN,NaN,NaN,25.5,17.733333,0.0,0.000120,0.000053,0.000053,0.000108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0,0,0,2,1,1,0,4.500000,5.779132,2.876154e+10,...,11.35,12.60,NaN,26.5,17.733333,0.2,0.000161,0.000082,0.000079,0.000160
596,0,0,0,2,1,46,0,4.500000,5.778906,2.875932e+10,...,9.25,NaN,NaN,25.5,17.733333,0.2,0.000034,0.000018,0.000018,0.000035
597,0,0,0,2,3,21,0,4.395000,5.779938,2.876943e+10,...,10.70,12.00,NaN,26.0,17.733333,0.2,0.000108,0.000052,0.000050,0.000103
598,0,0,0,2,3,27,0,4.365000,5.780190,2.877189e+10,...,8.05,9.35,10.6,25.0,17.733333,0.2,0.000247,0.000115,0.000115,0.000229
